In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset

In [3]:
data_spanish=load_dataset("spanish_billion_words",split="train")

Reusing dataset spanish_billion_words (/home/jhonparra/.cache/huggingface/datasets/spanish_billion_words/corpus/1.1.0/8ba50a854d61199f7d36b4c3f598589a2f8b493a2644b88ce80adb2cebcbc107)


In [4]:
from utils import clean_batch,homologate_accents

column_text="text"

def extract_all_chars(batch):
  all_text = " ".join(batch[column_text])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [6]:
sample_corpus=data_spanish.train_test_split(train_size=0.3)["train"]
sample_corpus.shape

(14077588, 1)

In [9]:
##preprocessing data

data_spanish=data_spanish.map(clean_batch,fn_kwargs={"text_column":column_text})
data_spanish=data_spanish.map(homologate_accents,fn_kwargs={"text_column":column_text})

  0%|          | 0/46925295 [00:00<?, ?ex/s]

  0%|          | 0/46925295 [00:00<?, ?ex/s]

### Push dataset to hub

In [ ]:
target_lang="es" 
data_spanish.push_to_hub(f"spanish_billion_words_clean", split="train")


In [ ]:
from datasets import load_dataset

username = "hf-test"  # change to your username

dataset_processed = load_dataset(f"jhonparra18/spanish_billion_words_clean", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset_processed["text"]))

In [3]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor,Wav2Vec2Processor,Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom")

model = Wav2Vec2ForCTC.from_pretrained("jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom")

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/309 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/260 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/309 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [6]:
model.push_to_hub("base-spanish-asr")

Upload file pytorch_model.bin:   0%|          | 32.0k/1.18G [00:00<?, ?B/s]

To https://huggingface.co/glob-asr/base-spanish-asr
   cb79256..969f11f  main -> main



'https://huggingface.co/glob-asr/base-spanish-asr/commit/969f11fa5a158fd8ab0f917d066aee35313d4973'